<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/Langchain_Knowledge_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain --progress-bar off
!pip install -qU langchainhub --progress-bar off
!pip install -qU openai --progress-bar off

In [2]:
# @title Connect to google drive
from google.colab import drive
import os
import json

drive.mount('/content/drive')

with open('/content/drive/MyDrive/env/env.json') as jsonfile:
    env = json.load(jsonfile)

Mounted at /content/drive


In [3]:
# @title Load llm
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="mistralai/mistral-7b-instruct:free",
  openai_api_key=env['openrouter.ai']['apiKey'],
  openai_api_base="https://openrouter.ai/api/v1",
  temperature= 0, max_tokens= 4096
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# Manual Graph creation

In [4]:
from langchain.prompts import PromptTemplate
from langchain.graphs.networkx_graph import KG_TRIPLE_DELIMITER
from langchain.indexes.prompts.knowledge_triplet_extraction import (
    KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT,
)
from langchain.chains import LLMChain

# We need to customize because of the model we are going to use.
# Prompt template for knowledge triple extraction
_DEFAULT_KNOWLEDGE_TRIPLE_EXTRACTION_TEMPLATE = (
    "<s>[INST]You are a networked intelligence helping a human track knowledge triples"
    " about all relevant people, things, concepts, etc. and integrating"
    " them with your knowledge stored within your weights"
    " as well as that stored in a knowledge graph."
    " Extract all of the knowledge triples base on user wrapped in ```."
    " A knowledge triple is a clause that contains a subject, a predicate,"
    " and an object. The subject is the entity being described,"
    " the predicate is the property of the subject that is being"
    " described, and the object is the value of the property.\n\n"
    " If you do your best work I'll tip you $10,000!\n\n"
    "EXAMPLE\n"
    "User: Nevada is a state in the US. It's also the number 1 producer of gold in the US.\n"
    f"Output: (Nevada, is a, state){KG_TRIPLE_DELIMITER}(Nevada, is in, US)"
    f"{KG_TRIPLE_DELIMITER}(Nevada, is the number 1 producer of, gold)\n"
    "END OF EXAMPLE\n\n"
    "EXAMPLE\n"
    "User: Descartes likes to drive antique scooters and play the mandolin.\n"
    f"Output: (Descartes, likes to drive, antique scooters){KG_TRIPLE_DELIMITER}(Descartes, plays, mandolin)\n"
    "END OF EXAMPLE\n\n"
    "User: ```{input}```\n"
    "Output: [/INST]"
)

KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT = PromptTemplate(
    input_variables=["input"],
    template=_DEFAULT_KNOWLEDGE_TRIPLE_EXTRACTION_TEMPLATE,
)

# Create an LLMChain using the knowledge triple extraction prompt
chain = LLMChain(llm=llm, prompt=KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT)

In [5]:
# Run the chain with the specified text
text = "The city of Paris is the capital and most populous city of France. The Eiffel Tower is a famous landmark in Paris."
triples = chain.invoke(
    {'input' : text}
)
triples

{'input': 'The city of Paris is the capital and most populous city of France. The Eiffel Tower is a famous landmark in Paris.',
 'text': '(Paris, is the capital of, France)\n<|>(Paris, is the most populous city of, France)\n<|>(Eiffel Tower, is a famous landmark in, Paris)'}

In [6]:
from pprint import pprint

def parse_triples(response, delimiter=KG_TRIPLE_DELIMITER):
    if not response:
        return []
    return response.split(delimiter)

triples_list = parse_triples(triples.get("text"))

pprint(triples_list)

['(Paris, is the capital of, France)\n',
 '(Paris, is the most populous city of, France)\n',
 '(Eiffel Tower, is a famous landmark in, Paris)']


# LLM Graph Transformer

- https://python.langchain.com/docs/use_cases/graph/constructing

In [7]:
!pip install -qU langchain-experimental
!pip install -qU langchain_fireworks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 6.4 MB/s eta 0:00:00


In [12]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_fireworks import ChatFireworks
from langchain.globals import set_llm_cache, set_debug


os.environ["FIREWORKS_API_KEY"] = env['fireworks.ai']['apiKey']

# We use ChatFireworks because we need model that can do function calling
llm_fireworks = ChatFireworks(model="accounts/fireworks/models/firefunction-v1", temperature=0, max_tokens=4000)


llm_transformer = LLMGraphTransformer(llm=llm_fireworks)

# Turn this on only if you want to debug other wise it's hard to see the conversations.
set_debug(True)

In [14]:
from langchain_core.documents import Document

text = """
Marie Curie, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "\nMarie Curie, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.\nShe was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.\nHer husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.\nShe was, in 1906, the first woman to become a professor at the University of Paris.\n"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "input": "\nMarie Curie, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.\nShe was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the onl